In [ ]:
# Linking with google drive
# simple upload to github, manual here: https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
%cd /content/drive/My Drive/time_series_analysis/time_series_prediction

/content/drive/My Drive/time_series_analysis/time_series_prediction


In [17]:
#!git clone https://{git_token}@github.com/{username}/{repository}

In [18]:
# trying to import python module as a separate script

In [19]:
import sys
sys.path.append('/content/drive/MyDrive/time_series_analysis/time_series_prediction/dataprocessing/')

In [39]:
import sys
sys.path.append('/content/drive/MyDrive/time_series_analysis/time_series_prediction/models/')

In [40]:
import datapreparation
import baselines

In [ ]:
#%load time_series_prediction/models/baselines.py

In [ ]:
#%load time_series_prediction/dataprocessing/datapreparation.py

In [21]:
import pandas as pd

In [22]:
sales_data = pd.read_csv("/content/sales.csv",   header = 0, sep = ";")
categories = pd.read_csv("/content/categories.csv",   header = 0, sep = ";")
traffic = pd.read_csv("/content/traffic.csv",   header = 0, sep = ";")

In [23]:
traffic['DateTime'] = traffic['week_starting_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
traffic.drop("week_starting_date", axis = 1, inplace = True)

traffic = traffic.set_index(["product_id", "DateTime"])
categories = categories.set_index(["product_id"])

In [24]:
# extended only to global max date horizon
# check nulls here because looks strange
sales_df_combined_right = datapreparation.prepare_sales_data(sales_data, categories, traffic, True, True, [1,2,3])

In [25]:
sales_df_combined_right

sales  category_id  ...  sales_lag_3W  traffic_lag_3W
product_id DateTime                        ...                              
1          2020-07-06    1.0           10  ...           0.0             0.0
           2020-07-13    0.0           10  ...           0.0             0.0
           2020-07-20    1.0           10  ...           0.0             0.0
           2020-07-27    3.0           10  ...           1.0             NaN
           2020-08-03    4.0           10  ...           0.0             NaN
...                      ...          ...  ...           ...             ...
3135       2020-11-30  102.0           43  ...          21.0           100.0
           2020-12-07   45.0           43  ...          29.0            81.0
           2020-12-14    8.0           43  ...         124.0           108.0
           2020-12-21   17.0           43  ...         102.0           142.0
           2020-12-28    7.0           43  ...          45.0            90.0

[217439 rows x 12 columns]

In [ ]:
sample = sales_df_combined_right.loc[[2658, 1308]]

In [ ]:
for i in all_items:
  print("*******************")
  print(i)
  tmp_dataset = sales_df_combined_right.loc[i]
  #n_splits_tmp = int(np.floor((tmp_dataset.shape[0] - 3)/2))
  try:
    tscv = TimeSeriesSplit(n_splits=5, test_size = 3)
    for tr_index, val_index in tscv.split(tmp_dataset):
      print (tr_index, val_index)
      X_tr, X_val = tmp_dataset[["sales"]].iloc[tr_index], tmp_dataset[["sales"]].iloc[val_index]
      print(X_tr)
      print(X_val)
      forecast = get_naive_forecast(X_tr.sales, horizon = 3)
      print(forecast)
      print(mean_absolute_percentage_error(X_val.sales, forecast))
  except Exception as e:
    print(e)

In [ ]:
# securing proper cross validation without errors

In [ ]:
# linear regression

In [ ]:
# hierarchical clustering

### Git commands

In [34]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Master_exploration.ipynb
	modified:   tests/baselines_tests.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [35]:
!git add .

In [36]:
!git config --global user.email "katarzyna.staniewicz0504@gmail.com"

In [37]:
!git commit -m 'Adding first unit tests'

[master 5db1070] Adding first unit tests
 2 files changed, 2 insertions(+), 2 deletions(-)
 rewrite tests/baselines_tests.ipynb (82%)


In [38]:
!git push

Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 1.45 KiB | 372.00 KiB/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/katarzynasornat/time_series_prediction
   9d8565b..5db1070  master -> master
